In [0]:
!pip install lightgbm
!pip install tsfresh
!pip install xgboost
!pip install catboost
!pip install shap
!pip install hyperopt

In [0]:
# The essentials
import pandas as pd
import numpy as np

from collections import defaultdict

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm

# Access our Google Drive
from google.colab import drive

# Gradient Boosting
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# TSFRESH Feature Extraction
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

from sklearn.model_selection import KFold, GridSearchCV

from collections import defaultdict, Counter
from scipy.stats import norm

from sklearn.preprocessing import PowerTransformer, StandardScaler

import shap

from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [0]:
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Rinse Over Run"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
20178.png
20451.png
20899.png
22112.png
22369.png
22414.png
22487.png
23011.png
23142.png
23599.png
23872.png
24804.png
24845.png
24872.png
25129.png
25908.png
25983.png
26270.png
27115.png
27243.png
27346.png
27366.png
27418.png
27508.png
all_train_preds_per_phase.p
baseline_features_with_preds_per_phase.csv
baseline_model_per_nunique_phases.csv
better_prev_object_id_per_10.csv
dtw_distances_3.p
extended_phase_predictors.csv
hcsta_features_3_3.csv
last_cleane

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_values.csv', index_col=0, parse_dates=['timestamp'])
test_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/test_values.csv', index_col=0, parse_dates=['timestamp'])
label_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_labels.csv', index_col='process_id')
all_data = pd.concat([train_df, test_df], axis=0)

train_df = train_df[train_df['phase'] != 'final_rinse']

train_df['phase_int'] = train_df['phase'].map({'pre_rinse': 1, 
                                               'caustic': 2, 
                                               'intermediate_rinse': 4, 
                                               'acid': 8})
test_df['phase_int'] = test_df['phase'].map({'pre_rinse': 1, 
                                             'caustic': 2, 
                                             'intermediate_rinse': 4, 
                                             'acid': 8})
train_process_combinations = pd.DataFrame(train_df.groupby('process_id')['phase_int'].unique().apply(lambda x: sum(x)))
test_process_combinations = pd.DataFrame(test_df.groupby('process_id')['phase_int'].unique().apply(lambda x: sum(x)))
process_combinations = pd.concat([train_process_combinations, test_process_combinations], axis=0)

recipe_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/recipe_metadata.csv', index_col='process_id')
recipe_df = recipe_df.drop('final_rinse', axis=1)
recipe_df['pre_rinse_num'] = recipe_df['pre_rinse'] * 1
recipe_df['caustic_num'] = recipe_df['caustic'] * 2
recipe_df['intermediate_rinse_num'] = recipe_df['intermediate_rinse'] * 4
recipe_df['acid_num'] = recipe_df['acid'] * 8
recipe_df['recipe'] = recipe_df['pre_rinse_num'] + recipe_df['caustic_num'] + recipe_df['intermediate_rinse_num'] + recipe_df['acid_num']

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
ts_real = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value',
    'flow_diff',
    'supply_flow_log',
    'return_flow_log'
]

# variables we'll use to create our time series features
ts_cols = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value',
    'flow_diff',
    #'supply_flow_log',
    #'return_flow_log'
]

# variables for binary time series features
bin_cols = [
    'supply_pump',
    'supply_pre_rinse',
    'supply_caustic',
    'return_caustic',
    'supply_acid',
    'return_acid',
    'supply_clean_water',
    'return_recovery_water',
    'return_drain',
    'object_low_level',
    'tank_lsh_caustic',
    'tank_lsh_acid',
    'tank_lsh_clean_water',
    'tank_lsh_pre_rinse'
]

process_comb_to_phases = {
    15: ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid'],
    3:  ['pre_rinse', 'caustic'],
    7:  ['pre_rinse', 'caustic', 'intermediate_rinse'],
    1:  ['pre_rinse'],
    8:  ['acid'],
    2:  ['caustic'],
    6:  ['caustic', 'intermediate_rinse'],
    14: ['caustic', 'intermediate_rinse', 'acid'],
}

# phases, ordered from earliest to latest
phases = ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid']

def encode_categorical(df):
    # Currently just copy-pasted from http://drivendata.co/blog/rinse-over-run-benchmark/
    
    # select process_id and pipeline
    meta = df[['process_id', 'pipeline', 'object_id']].drop_duplicates().set_index('process_id') 
    meta['object_id'] = meta['object_id'] // 10
    
    # convert categorical pipeline data to dummy variables
    meta = pd.get_dummies(meta, columns=['pipeline', 'object_id'])
    
    # pipeline L12 not in test data (so useless feature)
    if 'pipeline_L12' in meta:
        meta = meta.drop('pipeline_L12', axis=1)
    
    return meta
  
def count_zeros(x):
  return np.sum(x == 0)
  
def encode_real_timeseries(df):   
    ts_df = df[['process_id'] + ts_cols].set_index('process_id')
    
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', 
                                                   'count', 'median', 'sum', 
                                                   lambda x: x.tail(5).mean(),
                                                   count_zeros])
    
    cols = []
    for col in ts_features.columns:
        cols.append('real_{}'.format(col))
    ts_features.columns = cols
    
    return ts_features

def encode_binary_timeseries(df):
    ts_df = df[['process_id'] + bin_cols].set_index('process_id')
            
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['mean', 'std', 
                                                   lambda x: x.tail(5).mean(),
                                                   count_zeros])
    
    cols = []
    for col in ts_features.columns:
        cols.append('bin_{}'.format(col))
    ts_features.columns = cols
    
    return ts_features
  
def get_tsfresh_features(df):
    extraction_settings = EfficientFCParameters()
    filtered_funcs = ['abs_energy', 'mean_abs_change', 'mean_change', 
                      'skewness', 'kurtosis', 'absolute_sum_of_changes', 
                      'longest_strike_below_mean', 'longest_strike_above_mean', 
                      'count_above_mean', 'count_below_mean', 'last_location_of_maximum', 
                      'first_location_of_maximum', 'last_location_of_minimum', 
                      'first_location_of_minimum', 
                      'percentage_of_reoccurring_datapoints_to_all_datapoints', 
                      'percentage_of_reoccurring_values_to_all_values', 
                      'sum_of_reoccurring_values', 'sum_of_reoccurring_data_points', 
                      'ratio_value_number_to_time_series_length', 'maximum', 'minimum', 
                      'cid_ce', 'symmetry_looking', 'large_standard_deviation', 'quantile', 
                      'autocorrelation', 'number_peaks', 'binned_entropy', 'index_mass_quantile', 
                      'linear_trend',  'number_crossing_m']
#     new_funcs = ['augmented_dickey_fuller', 'number_cwt_peaks', 'agg_autocorrelation',
#                'spkt_welch_density', 'friedrich_coefficients', 'max_langevin_fixed_point',
#                'c3', 'ar_coefficient', 'mean_second_derivative_central', 'ratio_beyond_r_sigma',
#                'energy_ratio_by_chunks', 'partial_autocorrelation',
#                'fft_aggregated', 'time_reversal_asymmetry_statistic', 'range_count']
#     filtered_funcs += new_funcs
    filtered_settings = {}
    for func in filtered_funcs:
      filtered_settings[func] = extraction_settings[func]

    ts_features = extract_features(df[['process_id', 'timestamp', 'return_turbidity', 'return_flow', 'supply_flow', 'target_value', 'flow_diff']], 
                                   column_id='process_id', column_sort="timestamp", 
                                   column_kind=None, column_value=None,
                                   impute_function=impute, 
                                   default_fc_parameters=filtered_settings,
                                   show_warnings=False)
  
    return ts_features
                                       

def create_feature_matrix(df, processes, phases):
    df['return_flow'] = df['return_flow'].apply(lambda x: max(x, 0))
    df['supply_flow'] = df['supply_flow'].apply(lambda x: max(x, 0))
    df['target_value'] = df['return_flow'] * df['return_turbidity']
    df['flow_diff'] = df['supply_flow'] - df['return_flow']
    
    phase_data = df[(df['process_id'].isin(processes)) &
                    ((df['phase'].isin(phases)))]
    
    metadata = encode_categorical(phase_data)
    time_series = encode_real_timeseries(phase_data)
    binary_features = encode_binary_timeseries(phase_data)
    
    if len(phases) > 1:
      last_phase_data = phase_data[phase_data['phase'] == phases[-1]]
      time_series_last_phase = encode_real_timeseries(last_phase_data)
      new_cols = []
      for col in time_series_last_phase.columns:
        new_cols.append('last_{}'.format(col))
      time_series_last_phase.columns = new_cols
      binary_features_last_phase = encode_binary_timeseries(last_phase_data)
      new_cols = []
      for col in binary_features_last_phase.columns:
        new_cols.append('last_{}'.format(col))
      binary_features_last_phase.columns = new_cols
    
    tsfresh_features = get_tsfresh_features(phase_data)
    
    # join metadata and time series features into a single dataframe
    feature_matrix = metadata
    feature_matrix = feature_matrix.merge(time_series, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(binary_features, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(tsfresh_features, left_index=True, right_index=True)
    
    if len(phases) > 1:
      feature_matrix = feature_matrix.merge(time_series_last_phase, left_index=True, right_index=True)
      feature_matrix = feature_matrix.merge(binary_features_last_phase, left_index=True, right_index=True)
    
    return feature_matrix
    
  
def get_processes(data, phases, train=True):
    filtered_processes = []
    phases = set(phases)
    processes = set(data['process_id'])
    for process in processes:
        process_phases = set(data[data['process_id'] == process]['phase'])
        if train:
            if phases.issubset(process_phases):
                filtered_processes.append(process)
        else:
            if len(phases) == len(process_phases) == len(phases.intersection(process_phases)):
                filtered_processes.append(process)
    return filtered_processes

In [0]:
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, targets, weight):
        return custom_mape(np.exp(approxes), np.exp(targets)), len(targets)

In [0]:
#from tsfresh.feature_selection.relevance import calculate_relevance_table

def get_corr_features(X):
  row_idx, col_idx = np.where(X.corr() == 1)
  self_corr = set([(i, i) for i in range(X.shape[1])])
  return set(list(zip(row_idx, col_idx))) - self_corr 

def get_uncorr_features(data):
  X_train_corr = data.copy()
  correlated_features = get_corr_features(X_train_corr)
  
  corr_cols = set()
  for row_idx, col_idx in correlated_features:
    corr_cols.add(row_idx)
    corr_cols.add(col_idx)
  
  uncorr_cols = list(set(X_train_corr.columns) - set(X_train_corr.columns[list(corr_cols)]))
   
  col_mask = [False]*X_train_corr.shape[1]
  for col in corr_cols:
    col_mask[col] = True
  X_train_corr = X_train_corr.loc[:, col_mask]
  
  correlated_features = get_corr_features(X_train_corr)
  
  while correlated_features:
    print('{} correlated feature pairs left...'.format(len(correlated_features)))
    corr_row, corr_col = correlated_features.pop()
    col_mask = [True]*X_train_corr.shape[1]
    col_mask[corr_row] = False
    X_train_corr = X_train_corr.loc[:, col_mask]
    correlated_features = get_corr_features(X_train_corr)
  return list(set(list(X_train_corr.columns) + uncorr_cols))

def remove_features(data, target, p_val=0.25):
  single_cols = list(data.columns[data.nunique() == 1])
  
  uncorr_cols = get_uncorr_features(data)
  corr_cols = list(set(data.columns) - set(uncorr_cols))
  
  return list(set(single_cols + corr_cols))

In [0]:
def xgb_mape_eval(preds, dtrain):
    labels = dtrain.get_label()
    return 'mape', np.mean(np.abs((np.exp(labels) - np.exp(preds)) / np.maximum(290000, np.exp(labels))))


def xgb_quantile_obj(preds, labels, quantile=0.25):
    """
    Computes first-order derivative of quantile
    regression loss and a non-degenerate
    substitute for second-order derivative.
    Substitute is returned instead of zeros,
    because XGBoost requires non-zero
    second-order derivatives. See this page:
    https://github.com/dmlc/xgboost/issues/1825
    to see why it is possible to use this trick.
    However, be sure that hyperparameter named
    `max_delta_step` is small enough to satisfy:
    ```0.5 * max_delta_step <=
       min(quantile, 1 - quantile)```.
    @type preds: numpy.ndarray
    @type dmatrix: xgboost.DMatrix
    @type quantile: float
    @rtype: tuple(numpy.ndarray)
    """
    try:
        assert 0 <= quantile <= 1
    except AssertionError:
        raise ValueError("Quantile value must be float between 0 and 1.")

    labels = np.array(labels)
    preds = np.array(preds)
    errors = preds - labels

    left_mask = errors < 0
    right_mask = errors > 0

    grad = quantile * left_mask - (1 - quantile) * right_mask
    hess = np.ones_like(preds)

    return grad, hess
  
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, targets, weight):
        return custom_mape(np.exp(approxes), np.exp(targets)), len(targets)

In [1]:
combinations_per_recipe = {
    3: [3], #1, 2, 
    9: [8],
    15: [1, 3, 7, 15] # 2, 6, 14
}

all_mapes = defaultdict(list)
import warnings; warnings.filterwarnings('ignore')
for recipe in [15]:
    recipe_train_data = train_df[train_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    recipe_test_data = test_df[test_df['process_id'].isin(recipe_df[recipe_df['recipe'] == recipe].index)]
    for process_combination in combinations_per_recipe[recipe]:
      print('Recipe = {} || Combination = {}'.format(recipe, process_combination))
      train_processes = get_processes(recipe_train_data, process_comb_to_phases[process_combination])
      phase_features = create_feature_matrix(train_df, train_processes, process_comb_to_phases[process_combination])
      
      X = phase_features.loc[train_processes]
      y = np.log(label_df.loc[X.index]['final_rinse_total_turbidity_liter'])
    
      to_drop = remove_features(X, y)
      X = X.drop(to_drop, axis=1)
    
      kf = KFold(n_splits=5, random_state=2019)
      mapes = []
      shaps = []
      for train_idx, test_idx in kf.split(X, y):
        X_train = X.iloc[train_idx, :]
        X_test = X.iloc[test_idx, :]

        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]
            
        train_idx = np.random.choice(X_train.index, replace=False, size=int(0.9 * len(X_train)))
        val_idx = list(set(X_train.index) - set(train_idx))

        X_val = X_train.loc[val_idx, :]
        y_val = y_train.loc[val_idx]
        X_train = X_train.loc[train_idx, :]
        y_train = y_train.loc[train_idx]

        lgbm = LGBMRegressor(n_estimators=100000, objective='mape')
        lgbm.fit(X_train.values, y_train.values, eval_set=(X_val.values, y_val.values), early_stopping_rounds=100, verbose=50)
        
        lgbm_predictions = np.exp(lgbm.predict(X_test.values))
        mape = custom_mape(lgbm_predictions, np.exp(y_test))
        print('LightGBM TEST MAPE = {}'.format(mape))
        mapes.append(mape)
        all_mapes[(recipe, process_combination)].append(mape)
        
        plt.figure()
        plt.scatter(x=np.log(lgbm_predictions), y=y_test)
        plt.title('LightGBM y vs y_hat')
        plt.show()
        
        xgb = XGBRegressor(n_estimators=100000, objective=xgb_mape_eval)
        xgb.fit(X_train.values, y_train.values, eval_set=[(X_val.values, y_val.values)], early_stopping_rounds=100, verbose=50, eval_metric=xgb_mape_eval)

        xgb_predictions = np.exp(xgb.predict(X_test.values))
        mape = custom_mape(xgb_predictions, np.exp(y_test))
        print('XGBoost TEST MAPE = {}'.format(mape))
        mapes.append(mape)
        all_mapes[(recipe, process_combination)].append(mape)

        plt.figure()
        plt.scatter(x=np.log(xgb_predictions), y=y_test)
        plt.title('XGBoost y vs y_hat')
        plt.show()
        
        cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100, learning_rate=0.33,
                              loss_function='MAPE', eval_metric='MAPE', task_type='GPU')
        cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=50)
        
        cat_predictions = np.exp(cat.predict(X_test))
        mape = custom_mape(cat_predictions, np.exp(y_test))
        print('Catboost TEST MAPE = {}'.format(mape))
        mapes.append(mape)
        all_mapes[(recipe, process_combination)].append(mape)
        
        plt.figure()
        plt.scatter(x=np.log(cat_predictions), y=y_test)
        plt.title('Catboost y vs y_hat')
        plt.show()
        
        df = pd.DataFrame()
        df['LGBM'] = np.log(lgbm_predictions)
        df['XGB'] = np.log(xgb_predictions)
        df['CAT'] = np.log(cat_predictions)
        df['Mean'] = df[['LGBM', 'XGB', 'CAT']].mean(axis=1)
        df['Label'] = y_test.values

        plt.figure()
        pd.plotting.scatter_matrix(df)
        plt.show()

      print('Combination = {}, MAPE = {}+-{}'.format(process_combination, np.mean(mapes), np.std(mapes)))
      
    print('Recipe {}: MAPES: {}'.format(recipe, all_mapes))
    
for k in all_mapes:
    print(k, np.mean(all_mapes[k]), np.std(all_mapes[k]))


NameError: ignored

In [0]:
def objective(params):
    params = {
        'depth': int(params['depth']),
        'l2_leaf_reg': int(params['l2_leaf_reg']),
        'bagging_temperature': int(params['bagging_temperature']),
        'random_strength': int(params['random_strength']),
    }
    
    print(params)
    
    mapes = []
    shaps = []
    kf = KFold(n_splits=3)
    for fold_ix, (train_idx, test_idx) in enumerate(kf.split(X, y)):
      X_train = X.iloc[train_idx, :]
      X_test = X.iloc[test_idx, :]

      y_train = y.iloc[train_idx]
      y_test = y.iloc[test_idx]

      train_idx = np.random.choice(X_train.index, replace=False, size=int(0.9 * len(X_train)))
      val_idx = list(set(X_train.index) - set(train_idx))

      X_val = X_train.loc[val_idx, :]
      y_val = y_train.loc[val_idx]
      X_train = X_train.loc[train_idx, :]
      y_train = y_train.loc[train_idx]

      cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100, learning_rate=0.33,
                            loss_function='MAPE', eval_metric='MAPE', task_type='GPU', **params)
      cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=500)
        
      predictions = np.exp(cat.predict(X_test.values))
      mape = custom_mape(predictions, np.exp(y_test))
      print('Fold #{}: {}'.format(fold_ix+1, mape))
      mapes.append(mape)
      
    return np.mean(mapes)
      
      

space = {
    'depth': hp.uniform('depth', 2, 8),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 0, 50),
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 5),
    'random_strength': hp.uniform('random_strength', 0, 5),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

{'depth': 6, 'l2_leaf_reg': 32, 'bagging_temperature': 4, 'random_strength': 3}
0:	learn: 0.9981331	test: 0.9981484	best: 0.9981484 (0)	total: 71.9ms	remaining: 1h 59m 51s
500:	learn: 0.1128389	test: 0.1164191	best: 0.1164191 (500)	total: 21.6s	remaining: 1h 11m 34s
1000:	learn: 0.0314103	test: 0.0407169	best: 0.0407169 (1000)	total: 55.3s	remaining: 1h 31m 10s
1500:	learn: 0.0257539	test: 0.0391161	best: 0.0390993 (1493)	total: 1m 29s	remaining: 1h 38m 10s
bestTest = 0.03873542801
bestIteration = 1878
Shrink model to first 1879 iterations.
Fold #1: 0.3041592524857907
0:	learn: 0.9981413	test: 0.9981281	best: 0.9981281 (0)	total: 52.5ms	remaining: 1h 27m 31s
500:	learn: 0.1106859	test: 0.1063196	best: 0.1063196 (500)	total: 18.3s	remaining: 1h 43s
1000:	learn: 0.0292270	test: 0.0362388	best: 0.0362388 (1000)	total: 52.2s	remaining: 1h 26m 4s
1500:	learn: 0.0237593	test: 0.0345541	best: 0.0345244 (1456)	total: 1m 26s	remaining: 1h 34m 31s
2000:	learn: 0.0204099	test: 0.0341646	best: 0.0

In [0]:
best

{'bagging_temperature': 2.0806682444558398,
 'depth': 7.930167560093694,
 'l2_leaf_reg': 25.513833220225628,
 'random_strength': 2.59819372778521}